In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/'My Drive'/SemanticSegmentation

/content/gdrive/My Drive/SemanticSegmentation


In [0]:
import os
import numpy as np
from PIL import Image

In [0]:
from matplotlib.pyplot import imshow
%matplotlib inline

In [0]:
import numpy as np
import re
import random
from glob import glob
import shutil
import time

In [0]:
import tensorflow as tf

In [0]:
BACKGROUND_COLOR = [255, 0, 0]

In [0]:
def one_hot(gt_image):
  gt_bg = np.all(gt_image == BACKGROUND_COLOR, axis=2)
  gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
  gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)
  
  return gt_image

In [0]:
def preprocess_image(image, gt_image, width, height):
  
  image = image.resize((width, height), Image.ANTIALIAS)    # best down-sizing filter
  gt_image = gt_image.resize((width, height), Image.ANTIALIAS)    # best down-sizing filter
  
  image = np.float32(image)
  image = image/255.0
  
  gt_image = np.array(gt_image)
  gt_image = np.float32(one_hot(gt_image))
  
  return image, gt_image
  

In [0]:
def gen_batch_fn(data_dir, width = 1200, height = 350):
  
  image_paths = glob(os.path.join(data_dir, 'image_2', '*.png'))        
  gt_paths = {
      re.sub(r'_(lane|road)_', '_', os.path.basename(path)): path 
      for path in glob(os.path.join(data_dir, 'gt_image_2', '*_road_*.png'))
  }
  
  background_color = np.array([255, 0, 0])
  
  def get_batch(batch_size):
    
    random.shuffle(image_paths)
    for batch in range(0, len(image_paths), batch_size):
      images = []
      gt_images = []
      
      for image_file in image_paths[batch: batch + batch_size]:
        gt_file = gt_paths[os.path.basename(image_file)]
        
        image = Image.open(image_file)
        gt_image = Image.open(gt_file)
        
        image, gt_image = preprocess_image(image, gt_image, width, height)
        
        #print(image.shape, gt_image.shape)
        
        images.append(image)
        gt_images.append(gt_image)
        
      images = np.array(images)
      gt_images = np.array(gt_images) 
      
      yield images, gt_images
        
  
  return get_batch

In [0]:
get_batch = gen_batch_fn('data/data_road/training', width = 1120, height = 256)

In [0]:
for ims, gt_ims in get_batch(4):
  print(ims.shape, gt_ims.shape)
  #print(gt_ims)
  
  break

(4, 256, 1120, 3) (4, 256, 1120, 2)


In [0]:
def save_test_predictions(sess, test_dir, save_dir, width = 1120, height = 256):
  output_dir = os.path.join(save_dir, str(time.time()))
  if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
  os.makedirs(output_dir)
  
  print("Training finished. Saving images to {}".format(output_dir))
  
  for image in glob(os.path.join(test_dir, 'image_2', '*.png')):
    im = Image.open(image)
    im = im.resize((width, height), Image.ANTIALIAS)
    im_arr = np.array(im)
    
    im_scaled = np.float32(im_arr)/255.0
    
    label = sess.run([tf.math.softmax(output_label)], feed_dict = {input_im: [im_scaled]})
    print(len(label))
    label = label[0][:, :, 1]
    segmentation = (label > 0.5).reshape(height, width, 1)
    
    mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))    
    mask = Image.fromarray(mask, 'RGBA')
    
    Image.paste(im, mask = mask)
    
    im.save(os.path.join(save_dir, os.path.basename(image)))
    
    break
    

In [0]:

from tensorflow.contrib import slim
from tensorflow.contrib.slim.nets import resnet_v2
#from models import resnet_v2

In [0]:
from tensorflow.keras import layers

In [0]:
tf.reset_default_graph()

In [0]:
def arm_module(layer, n_filter_maps):
  
  net = tf.math.reduce_mean(layer, axis = [1,2], keepdims = True, name = 'global_avg_pool')
  net = layers.Conv2D(filters = n_filter_maps, kernel_size = [1,1])(net)
  net = layers.BatchNormalization()(net)
  net = tf.math.sigmoid(net, name = 'sigmoid')
  
  scaled_layer = tf.multiply(layer, net)
  
  return scaled_layer

In [0]:
def ffm_module(spatial_layer, context_layer, num_classes):
  
  input_features = tf.concat([spatial_layer, context_layer], axis = -1)
  net = layers.Conv2D(filters = num_classes, kernel_size = [3,3], padding = 'same', activation = 'relu')(input_features)
  net = layers.BatchNormalization()(net)
  
  net_vector = tf.reduce_mean(net, axis = [1,2], keepdims = True)
  
  # First 1x1 convolution uses 16 filters
  net_vector = layers.Conv2D(filters = 16, kernel_size = [1,1], padding = 'same', activation = 'relu')(net_vector)
  net_vector = layers.Conv2D(filters = num_classes, kernel_size = [1,1], padding = 'same')(net_vector)
  net_vector = layers.Activation('sigmoid')(net_vector)
  
  net_scaled = tf.multiply(net, net_vector)
  net = tf.add(net, net_scaled)
  
  return net
  

In [0]:
frontend = 'ResNet101'
num_classes = 2
input_im = tf.placeholder(shape = (None, 256, 1120, 3), dtype = tf.float32)
gt_im = tf.placeholder(shape = (None, 256, 1120, 2), dtype = tf.float32)

In [0]:
## Spatial Path

In [0]:
layer_spatial = layers.Conv2D(input_shape=(None, 1120, 256, 3), filters=64, kernel_size=[3,3], strides=2, padding='same', activation='relu')(input_im)
layer_spatial = layers.Conv2D(filters=128, kernel_size=[3,3], strides=2, padding='same', activation='relu')(layer_spatial)
spatial_output = layers.Conv2D(filters=256, kernel_size=[3,3], strides=2, padding='same', activation='relu')(layer_spatial)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
spatial_output

<tf.Tensor 'conv2d_2/Relu:0' shape=(?, 32, 140, 256) dtype=float32>

In [0]:
## Context Path

In [0]:
with slim.arg_scope(resnet_v2.resnet_arg_scope()):
  #tf.reset_default_graph()
  last_layer, end_points = resnet_v2.resnet_v2_101(input_im, is_training=True, scope='resnet_v2_101', global_pool = False)
  frontend_scope='resnet_v2_101'
  init_fn = slim.assign_from_checkpoint_fn(model_path=os.path.join('models', 'resnet_v2_101.ckpt'), var_list=slim.get_model_variables('resnet_v2_101'), ignore_missing_vars=True)

In [0]:
layer_reduced16 = end_points[frontend_scope + '/block2']
layer_reduced32 = last_layer

In [0]:
layer_arm16 = arm_module(layer_reduced16, n_filter_maps = 512)
layer_arm32 = arm_module(layer_reduced32, n_filter_maps = 2048)
layer_global_context = tf.reduce_mean(last_layer, axis = [1,2], keepdims = True, name = 'global_context')

In [0]:
## Combining Context Features
layer_context1 = tf.math.multiply(layer_arm32, layer_global_context)
layer_context1 = layers.UpSampling2D(size = 4, interpolation = 'bilinear')(layer_context1)
layer_context2 = layers.UpSampling2D(size = 2, interpolation = 'bilinear')(layer_arm16)

context_output = tf.concat([layer_context1, layer_context2], axis = -1)

In [0]:
print(context_output)

Tensor("concat:0", shape=(?, 32, 140, 2560), dtype=float32)


In [0]:
ffm_output = ffm_module(spatial_output, context_output, num_classes)

In [0]:
output_label = layers.UpSampling2D(size = 8, interpolation = 'bilinear')(ffm_output)
output_label = layers.Conv2D(filters = num_classes, kernel_size = [1,1], activation = None)(output_label)

In [0]:
print(output_label)

Tensor("conv2d_8/BiasAdd:0", shape=(?, 256, 1120, 2), dtype=float32)


In [0]:
EPOCHS = 100
BATCH_SIZE = 16
TEST_DIR = 'data/data_road/testing'
SAVE_DIR = 'saved_tests'

In [0]:
cross_entropy_loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(onehot_labels = gt_im, logits = output_label))
optimizer = tf.train.RMSPropOptimizer(learning_rate = 0.0001, decay = 0.995).minimize(cross_entropy_loss)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [0]:
sess = tf.Session()

In [0]:
#with tf.Session() as sess:
sess.run(tf.global_variables_initializer())
init_fn(sess)

In [0]:
get_batch = gen_batch_fn('data/data_road/training', width = 1120, height = 256)
for i in range(EPOCHS):
  print("EPOCH {} ...".format(i+1))
  for image, label in get_batch(BATCH_SIZE):
      _, loss = sess.run([optimizer, cross_entropy_loss], 
                         feed_dict={input_im: image, gt_im: label})
      print("Loss: = {:.3f}".format(loss))
  print()


#save_test_predictions(sess, TEST_DIR, SAVE_DIR, width = 1120, height = 256)


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/resnet_v2_101.ckpt
EPOCH 1 ...
Loss: = 0.697
Loss: = 0.708
Loss: = 0.713
Loss: = 0.701
Loss: = 0.701
Loss: = 0.710
Loss: = 0.701
Loss: = 0.691
Loss: = 0.698
Loss: = 0.696
Loss: = 0.686
Loss: = 0.689
Loss: = 0.683
Loss: = 0.684
Loss: = 0.681
Loss: = 0.678
Loss: = 0.681
Loss: = 0.684
Loss: = 0.669

EPOCH 2 ...
Loss: = 0.675
Loss: = 0.667
Loss: = 0.643
Loss: = 0.662
Loss: = 0.654
Loss: = 0.653
Loss: = 0.655
Loss: = 0.641
Loss: = 0.628
Loss: = 0.621
Loss: = 0.658
Loss: = 0.636
Loss: = 0.604
Loss: = 0.626
Loss: = 0.604
Loss: = 0.605
Loss: = 0.610
Loss: = 0.592
Loss: = 0.532

EPOCH 3 ...
Loss: = 0.595
Loss: = 0.590
Loss: = 0.580
Loss: = 0.555
Loss: = 0.551
Loss: = 0.565
Loss: = 0.583
Loss: = 0.553
Loss: = 0.595
Loss: = 0.533
Loss: = 0.558
Loss: = 0.552
Loss: = 0.561
Loss: = 0.552
Loss: = 0.565
Loss: = 0.530
Loss: = 0.550
Loss: = 0.553
Loss: = 0.550

EPOCH 4 

In [0]:
save_dir = SAVE_DIR
test_dir = TEST_DIR
width = 1120
height = 256

In [0]:
output_dir = os.path.join(save_dir, str(time.time()))
if os.path.exists(output_dir):
  shutil.rmtree(output_dir)
os.makedirs(output_dir)

print("Training finished. Saving images to {}".format(output_dir))

for image in glob(os.path.join(test_dir, 'image_2', '*.png')):
  im = Image.open(image)
  im = im.resize((width, height), Image.ANTIALIAS)

  im_scaled = np.float32(im)/255.0
  
  inputs = np.array([im_scaled])
    
  label = sess.run(tf.math.softmax(output_label), feed_dict = {input_im: inputs})
  test_label = label
  #print(np.array(label).shape)
  label = label[0][:, :, 1]
  segmentation = (label > 0.5).reshape(height, width, 1)

  #mask = 
  mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
  mask = mask.astype('uint8')
  mask = Image.fromarray(mask, 'RGBA')

  im.paste(mask, mask = mask)

  im.save(os.path.join(output_dir, os.path.basename(image)))

In [0]:
get_batch = gen_batch_fn('data/data_road/training', width = 1120, height = 256)
for i in range(EPOCHS):
  print("EPOCH {} ...".format(i+301))
  for image, label in get_batch(BATCH_SIZE):
      _, loss = sess.run([optimizer, cross_entropy_loss], 
                         feed_dict={input_im: image, gt_im: label})
      print("Loss: = {:.3f}".format(loss))
  print()


#save_test_predictions(sess, TEST_DIR, SAVE_DIR, width = 1120, height = 256)

EPOCH 301 ...
Loss: = 0.036
Loss: = 0.031
Loss: = 0.021
Loss: = 0.022
Loss: = 0.022
Loss: = 0.021
Loss: = 0.022
Loss: = 0.027
Loss: = 0.023
Loss: = 0.021
Loss: = 0.024
Loss: = 0.020
Loss: = 0.026
Loss: = 0.022
Loss: = 0.023
Loss: = 0.026
Loss: = 0.026
Loss: = 0.025
Loss: = 0.049

EPOCH 302 ...
Loss: = 0.045
Loss: = 0.023
Loss: = 0.020
Loss: = 0.019
Loss: = 0.026
Loss: = 0.024
Loss: = 0.024
Loss: = 0.024
Loss: = 0.022
Loss: = 0.026
Loss: = 0.026
Loss: = 0.025
Loss: = 0.020
Loss: = 0.024
Loss: = 0.018
Loss: = 0.026
Loss: = 0.021
Loss: = 0.021
Loss: = 0.031

EPOCH 303 ...
Loss: = 0.021
Loss: = 0.016
Loss: = 0.025
Loss: = 0.020
Loss: = 0.024
Loss: = 0.022
Loss: = 0.021
Loss: = 0.026
Loss: = 0.022
Loss: = 0.022
Loss: = 0.022
Loss: = 0.020
Loss: = 0.024
Loss: = 0.024
Loss: = 0.025
Loss: = 0.022
Loss: = 0.021
Loss: = 0.022
Loss: = 0.029

EPOCH 304 ...
Loss: = 0.024
Loss: = 0.022
Loss: = 0.020
Loss: = 0.021
Loss: = 0.023
Loss: = 0.018
Loss: = 0.022
Loss: = 0.023
Loss: = 0.018
Loss: = 0.027
Los

In [0]:
print(end_points['pool4'])

Tensor("resnet_v2_101/block2/unit_4/bottleneck_v2/add:0", shape=(?, 70, 16, 512), dtype=float32)


In [0]:
# Equivalent to pool4
print(end_points[frontend_scope + '/block2'])

Tensor("resnet_v2_101/block2/unit_4/bottleneck_v2/add:0", shape=(?, 70, 16, 512), dtype=float32)


In [0]:
print(end_points['pool5'])

Tensor("resnet_v2_101/postnorm/Relu:0", shape=(?, 35, 8, 2048), dtype=float32)


In [0]:
# Equivalent to pool5
print(logits)

Tensor("resnet_v2_101/postnorm/Relu:0", shape=(?, 35, 8, 2048), dtype=float32)


In [0]:
print(end_points[frontend_scope + '/block3'])

Tensor("resnet_v2_101/block3/unit_23/bottleneck_v2/add:0", shape=(?, 35, 8, 1024), dtype=float32)


In [0]:
for key in end_points:
  print(key)

In [0]:
x = end_points[frontend_scope + '/block1']

In [0]:
print(x)

Tensor("resnet_v2_101/block1/unit_3/bottleneck_v2/add:0", shape=(?, ?, ?, 256), dtype=float32)


In [0]:
print(end_points['pool3'])

Tensor("resnet_v2_101/block1/unit_3/bottleneck_v2/add:0", shape=(?, ?, ?, 256), dtype=float32)


In [0]:
def main():
  num_classes = 2
  data_dir = './gdrive/My Drive/SemanticSegmentation/data'
